In [30]:
import fitz
# from PIL import Image 
import os
import json

In [31]:
pwd = os.getcwd()
source = os.path.join(pwd, 'app')
print(source)
file_1= os.path.join(source,'LLaVA.pdf')
file_2= os.path.join(source,'Interior.pdf')
print(file_1)
print(file_2)

/Users/jc/Desktop/main/multimodal_rag/app
/Users/jc/Desktop/main/multimodal_rag/app/LLaVA.pdf
/Users/jc/Desktop/main/multimodal_rag/app/Interior.pdf


In [32]:
#source_files : 현재 경로
#file_ : 이미지 추출할 pdf 경로
def load_pdf(source_files, file_, pdf_name):
	# open the file 
	pdf_file = fitz.open(file_) 
	image_counter = 0
	metadata = {}
	for page_index in range(0,len(pdf_file)): 
		# get the page itself 
		page = pdf_file[page_index] 
		# get block details from the page
		blocks = pdf_file[page_index].get_text("blocks")
		# get image info and title details 
		image_meta = [ (blocks[i][4], blocks[i+1][4]) for i in range(0,len(blocks)) if blocks[i][-1]==1 ] 	
		image_info= [ image_meta[0][0] if image_meta  else []] 
		image_title = [ image_meta[0][1] if image_meta else []] 
		# prepare image meta data from the page
		for image in page.get_images():
			image_id = image[7] # img<no>
			image_block_id = image[0] # block number 
			image_title_block_id = image_block_id+1 # image title block number
			image_dim = image[2],image[3] # image dimension details
			
			print(f"[+] Page:{page_index}, Image : {image_id}, Block:{image_block_id}, Image Dim:{image_dim}")
			image_counter = image_counter+1

			# Update metadata dictionary with image information
			metadata[image_counter] = {
				'page': page_index,
				'image': image_id,
				'block': image_block_id,
				'image_dim': image_dim,
				'image_info': str(image_info[0]),
				'image_title': str(image_title[0]),
				'image_file': f"{image_id}_{image_block_id}.png",
				'image_path': os.path.join(source_files, f"{image_id}_{image_block_id}.png")
			}
		# save the images to the local file system
			pix = fitz.Pixmap(pdf_file, image[0])
			# image file name contains image name 'img<no>' and block number
            #pix.save(os.path.join(source_files, f"{image_id}_{image_block_id}.png"))
			pix.save(os.path.join(source_files+'/'+pdf_name, f"{image_id}_{image_block_id}.png"))
	print(f"Total Images: {image_counter}")
	
	with open(os.path.join(source, f'metadata.json'),'w') as f:
		json.dump(metadata,f)
	return metadata

In [33]:
#source 경로 내 파일 리스트 생성
file_list = os.listdir(source)

#pdf 리스트 추출
pdf_list=[]
[pdf_list.append(x) for x in file_list if x[-3:]=='pdf']

#pdf명 파일 생성
for x in pdf_list:
    if x[-3:]=='pdf':
        if os.path.isdir(source+"/"+str(x[:-4])) == True :
            pass #파일이 존재하면 pass
        else:
            os.mkdir(source+"/"+str(x[:-4]))

#pdf에서 이미지 추출해서 pdf 파일명의 폴더에 저장
for i in pdf_list:
    load_pdf(source, source+"/"+i,str(i[:-4]))

[+] Page:0, Image : Im0, Block:726, Image Dim:(663, 268)
[+] Page:0, Image : Im1, Block:727, Image Dim:(600, 400)
[+] Page:1, Image : Im0, Block:141, Image Dim:(660, 375)
[+] Page:1, Image : Im0, Block:132, Image Dim:(660, 375)
[+] Page:2, Image : Im0, Block:206, Image Dim:(596, 398)
[+] Page:2, Image : Im0, Block:164, Image Dim:(596, 398)
[+] Page:2, Image : Im0, Block:150, Image Dim:(596, 398)
[+] Page:2, Image : Im0, Block:155, Image Dim:(596, 398)
[+] Page:2, Image : Im0, Block:203, Image Dim:(596, 398)
[+] Page:2, Image : Im0, Block:12, Image Dim:(596, 398)
[+] Page:2, Image : Im0, Block:160, Image Dim:(596, 398)
[+] Page:4, Image : Im0, Block:20, Image Dim:(1621, 1080)
[+] Page:5, Image : Im0, Block:23, Image Dim:(1620, 1080)
[+] Page:6, Image : Im0, Block:26, Image Dim:(1620, 1080)
[+] Page:8, Image : Im0, Block:32, Image Dim:(600, 400)
[+] Page:8, Image : Im1, Block:33, Image Dim:(1620, 1080)
[+] Page:9, Image : Im0, Block:36, Image Dim:(1621, 1080)
[+] Page:10, Image : Im0, Bl